# Training a digit classifier based on Chapter 4 of the fast.ai book

Chapter 4 gives an example for classifying 3s and 7s. Here I will train a model to classify all digits. 

In [52]:
!pip install fastai
from fastai.vision.all import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [53]:
!pip install fastbook
from fastbook import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 1. Load data
Let's download the MNIST dataset.

In [54]:
path = untar_data('https://s3.amazonaws.com/fast-ai-imageclas/mnist_png.tgz')
(path/'training').ls()

(#10) [Path('/root/.fastai/data/mnist_png/training/2'),Path('/root/.fastai/data/mnist_png/training/4'),Path('/root/.fastai/data/mnist_png/training/3'),Path('/root/.fastai/data/mnist_png/training/7'),Path('/root/.fastai/data/mnist_png/training/8'),Path('/root/.fastai/data/mnist_png/training/9'),Path('/root/.fastai/data/mnist_png/training/1'),Path('/root/.fastai/data/mnist_png/training/5'),Path('/root/.fastai/data/mnist_png/training/6'),Path('/root/.fastai/data/mnist_png/training/0')]

In [55]:
digit_paths = [(path/'training'/f'{o}').ls().sorted() for o in range(0,10)]
#Print a random 4
im4_path = digit_paths[4][1]
im4 = Image.open(im4_path)
im4

Let's turn the images into PyTorch tensors. 

In [56]:
digit_tensors = [[tensor(Image.open(o)) for o in digit_paths[i]] for i in range(0,10)]

Stack the list of digits into 10 separate tensors

In [57]:
stacked_digits = [torch.stack(digit_tensors[i]) for i in range(0,10)]

In [58]:
stacked_digits[4].size() 

torch.Size([5842, 28, 28])

To make sure that the pixel values are between 0 and 1 divide them by 255.

In [59]:
scaled_digits = [stacked_digits[i].float()/255 for i in range(0,10)]

In [60]:
scaled_digits[4].shape

torch.Size([5842, 28, 28])

### Set up training
1. Define the output, the training set and the test set
2. Define the classification and the loss function
3. Pick an example digit

In [113]:
#1. The training set
data_x = torch.cat([scaled_digits[i] for i in range(0,10)]).view(-1,28*28)
data_x.shape
#The outcomes
#Define the output as a vector of length 10 with 1 if the image is of that indices digit, 0 otherwise
tensor_id = torch.eye(10)
data_y = torch.cat([tensor_id[i].repeat(len(scaled_digits[i]),1) for i in range(0,10)])

dataset = list(zip(data_x, data_y))

train, test = torch.utils.data.random_split(dataset, lengths = [(int(len(dataset)*0.8)),len(dataset)-int(len(dataset)*0.8)])
train_x = torch.stack([train[i][0] for i in range(0,len(train))])
train_y = torch.stack([train[i][1] for i in range(0,len(train))])

test_x = torch.stack([test[i][0] for i in range(0,len(test))])
test_y = torch.stack([test[i][1] for i in range(0,len(test))])

torch.Size([12000, 10])

In [156]:
#2. The classification and the loss
#The image should be classified as the index with the highest element in the output
def classify(outp):  
    return torch.argmax(outp)

#The loss function should be the mean sum of squared difference between the prediction and the true output.
#High value at the wrong digit and small value at the right digit will be penalized
def loss(outp, prediction):
    return torch.sum((outp - prediction).square(), dim = 1).mean()

In [65]:
#3. Pick an example digit
example_digit = scaled_digits[4][1].view(-1,28*28)

In [66]:
#TODO: split off a validation dataset

## 2. Build a neural net model all the way from matrices
For the sake of simplicity I will use two layers with a sigmoid non-linarity. 

In [115]:
#1. Initialize the parameters
w = torch.randn(28*28,10).requires_grad_()
b = torch.randn(10).requires_grad_()

In [116]:
#2. Calculate the predictions

#Define the linear function (layer) 
def linear_layer(inp,w,b):
    return (inp@w) + b

#Prediction example
pred_vector = linear_layer(example_digit, w,b)
pred = pred_vector.argmax() 

#Calculate the prediction for all of train_x
predictions_train_x = torch.stack([linear_layer(dig, w,b) for dig in train_x])

In [117]:
#TODO: 3. Calculate the loss
loss_1 = loss(predictions_train_x, train_y)  
loss_1

tensor(1179.3499, grad_fn=<MeanBackward0>)

In [118]:
#4. Calculate the gradients
loss_1.backward()
w.grad
b.grad

tensor([-23.6789,  16.5147,  22.6215, -27.6663,  -4.4462,  10.0359,  -8.9582,   2.2399,  -0.4999,  14.3081])

In [119]:
#5. Step the weights
#Pick learning rate
lr = 1e-3
w.data -= lr*w.grad.data
b.data -= lr*b.grad.data
w.grad = None
b.grad = None

In [120]:
improved_predictions_train_x = torch.stack([linear_layer(dig, w,b) for dig in train_x])

In [121]:
loss_2 = loss(improved_predictions_train_x, train_y)
loss_2

tensor(1073.6483, grad_fn=<MeanBackward0>)

In [122]:
#Let's put all of this together
def initialize():
    w = torch.randn(28*28,10).requires_grad_()
    b = torch.randn(10).requires_grad_()
    return w,b

def step(w,b, lr):
    predictions_train_x = torch.stack([linear_layer(dig, w,b) for dig in train_x])
    loss_d = loss(predictions_train_x, train_y)
    print(loss_d)
    loss_d.backward()
    w.data -= lr*w.grad.data
    w.grad = None
    b.data -= lr*b.grad.data
    b.grad = None


In [123]:
w,b = initialize()
lr = 1e-3
for i in range(0,10):
    step(w,b,lr)

example_prediction = linear_layer(example_digit, w,b)
print(example_prediction)

tensor(707.6218, grad_fn=<MeanBackward0>)
tensor(672.1191, grad_fn=<MeanBackward0>)
tensor(641.5903, grad_fn=<MeanBackward0>)
tensor(615.2924, grad_fn=<MeanBackward0>)
tensor(592.5938, grad_fn=<MeanBackward0>)
tensor(572.9577, grad_fn=<MeanBackward0>)
tensor(555.9282, grad_fn=<MeanBackward0>)
tensor(541.1172, grad_fn=<MeanBackward0>)
tensor(528.1951, grad_fn=<MeanBackward0>)
tensor(516.8818, grad_fn=<MeanBackward0>)
tensor([[6.9709, 4.0084, 6.1093, 3.3838, 9.8887, 4.3656, 1.1545, 0.7734, 3.7606, 0.6269]], grad_fn=<AddBackward0>)


In [153]:
#Add a softmax function to normalize the output vetor. This is to obtain 'probabilities'
def step_sig(w,b, lr):
    predictions_train_x = torch.stack([torch.nn.functional.softmax(linear_layer(dig, w,b)) for dig in train_x])
    loss_d = loss(predictions_train_x, train_y)
    print(loss_d)
    loss_d.backward()
    w.data -= lr*w.grad.data
    w.grad = None
    b.data -= lr*b.grad.data
    b.grad = None


In [125]:
#import pytorch as nn
#torch.softmax(example_prediction, dim = 10)
torch.nn.functional.softmax(example_prediction, dim = 1)

tensor([[4.9693e-02, 2.5686e-03, 2.0995e-02, 1.3754e-03, 9.1935e-01, 3.6716e-03, 1.4801e-04, 1.0111e-04, 2.0048e-03, 8.7330e-05]], grad_fn=<SoftmaxBackward0>)

In [161]:
w,b = initialize()
lr = 0.1
for i in range(0,10):
    step_sig(w,b,lr)

example_prediction = torch.nn.functional.softmax(linear_layer(example_digit, w,b))
print(example_prediction)

<ipython-input-153-d77ed0369ab9>:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions_train_x = torch.stack([torch.nn.functional.softmax(linear_layer(dig, w,b)) for dig in train_x])


tensor(1.5419, grad_fn=<MeanBackward0>)
tensor(1.5398, grad_fn=<MeanBackward0>)
tensor(1.5378, grad_fn=<MeanBackward0>)
tensor(1.5357, grad_fn=<MeanBackward0>)
tensor(1.5337, grad_fn=<MeanBackward0>)
tensor(1.5317, grad_fn=<MeanBackward0>)
tensor(1.5298, grad_fn=<MeanBackward0>)
tensor(1.5278, grad_fn=<MeanBackward0>)
tensor(1.5259, grad_fn=<MeanBackward0>)
tensor(1.5240, grad_fn=<MeanBackward0>)
tensor([[2.2827e-13, 2.2182e-12, 3.3376e-01, 5.1007e-12, 4.1365e-05, 3.4366e-06, 4.7446e-07, 6.6619e-01, 2.0202e-11, 5.0161e-08]], grad_fn=<SoftmaxBackward0>)


<ipython-input-161-9327eecee8bf>:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  example_prediction = torch.nn.functional.softmax(linear_layer(example_digit, w,b))


In [162]:
#Test 
predictions_test = torch.stack([torch.nn.functional.softmax(linear_layer(dig, w,b)) for dig in test_x])
loss_test = loss(predictions_test, test_y)
loss_test

<ipython-input-162-087942e95b11>:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions_test = torch.stack([torch.nn.functional.softmax(linear_layer(dig, w,b)) for dig in test_x])


tensor(1.5260, grad_fn=<MeanBackward0>)

In [168]:
#Calculate accuracy (how many digits were classified correctly)
digit_predictions = predictions_test.argmax(dim = 1)
digit_predictions

(digit_predictions == test_y.argmax(dim = 1)).float().mean()
#This is very very very low accuracy. Need to invesitgate the reason for this. Is it the too simple neural net or an mistake in my code?

tensor(0.1379)

## 3. Build a neural net with PyTorch

In [196]:
#Load the data
dl = DataLoader(train, batch_size = 256)
valid_dl = DataLoader(test, batch_size=256)
dls = DataLoaders(dl,valid_dl)

In [197]:
#Set up the architecture
neural_net  = nn.Sequential(
    nn.Linear(28*28,40),
    nn.ReLU(),
    nn.Linear(40,10),
    nn.Softmax(dim=1)
)

In [198]:
learn = Learner(dls, neural_net, opt_func = SGD, loss_func = loss, metrics = loss)

In [199]:
learn.fit(5, lr = 0.01)

epoch,train_loss,valid_loss,loss,time
0,0.886724,0.882886,0.882886,00:01
1,0.866502,0.860105,0.860105,00:01
2,0.832094,0.821791,0.821791,00:01
3,0.781169,0.768481,0.768481,00:00
4,0.719882,0.705817,0.705817,00:00


In [80]:
#1. Initialize parameters and model
linear_model = nn.Linear(28*28,10)
w,b = linear_model.parameters()

In [81]:
#scaled_digits[4][1].squeeze

In [82]:
#2. Make prediction with model
linear_model(scaled_digits[4][1])

RuntimeError: ignored

## 3. Train a neural net with fast.ai